In [11]:
import torch
import torchvision
import cv2
import numpy as np
from sort.sort import *
from deep_sort_realtime.deepsort_tracker import DeepSort

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
# Load the pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)

# Set the model to evaluation mode
model.eval()

/home/ak232003/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ak232003/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [14]:
object_tracker = DeepSort(max_iou_distance=1, max_age=200,nms_max_overlap = 5)

In [15]:
COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

In [16]:
def detect_cars(img, model, threshold = 0.5):

    transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
    img1 = transform(img).to(device)

    with torch.no_grad():
        pred = model([img1])

    detections = []
    # pred = model(img)
    # car_pred = pred.pred[0][pred.pred[0][:, 5] == 2]
    # boxes = car_pred[:, :4].cpu().numpy()
    # confidences = car_pred[:, 4].cpu().numpy()
    # class_label = car_pred[:, 5].cpu().numpy()

    pred_class1 = [COCO_INSTANCE_CATEGORY_NAMES[i] for i in list(pred[0]['labels'].cpu().numpy())] # Get the Prediction Score
    pred_boxes1 = [[i[0], i[1], i[2], i[3]] for i in list(pred[0]['boxes'].detach().cpu().numpy())] # Bounding boxes
    pred_score1 = list(pred[0]['scores'].detach().cpu().numpy())
    pred_labels1 = list(pred[0]['labels'].detach().cpu().numpy())

    pred_class = []
    pred_boxes = []
    pred_score = []
    pred_labels = []

    for i in range(len(pred_class1)):
        if(pred_class1[i] == 'car'):
            pred_class.append(pred_class1[i])
            pred_boxes.append(pred_boxes1[i])
            pred_score.append(pred_score1[i])
            pred_labels.append(pred_labels1[i])

    for i in range(len(pred_labels)):
        if(pred_score[i] > threshold):
            cv2.rectangle(img, (int(pred_boxes[i][0]), int(pred_boxes[i][1])), (int(pred_boxes[i][2]), int(pred_boxes[i][3])), (0, 255, 0), 2)
            cv2.putText(img, "Car", (int(pred_boxes[i][0])-10, int(pred_boxes[i][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            detections.append((pred_boxes[i], pred_score[i]))
    
    return detections

In [17]:
def update_track(tracks, count, track_ids):
    for t in tracks:
        if not t.is_confirmed():
            continue
        t_id = t.track_id
        if(t_id not in track_ids):
            count += 1
            track_ids.append(t_id)
        
        ltrb = t.to_ltrb()
        bbox = ltrb

    return track_ids, count

In [18]:
cap = cv2.VideoCapture('./mp4/Test.mp4')

In [19]:
track_ids = []
count = 0 

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    detections = detect_cars(frame, model, 0.99)
    tracks = object_tracker.update_tracks(detections, frame=frame)
    track_ids, count = update_track(tracks, count, track_ids)
    cv2.putText(frame, f'Count: {int(count)}', (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)

    # Display result
    cv2.imshow('result', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and destroy windows
cap.release()
cv2.destroyAllWindows()
    

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is not the object's thread (0x78e0cdc0).
Cannot move to target thread (0x7b8aded0)

QObject::moveToThread: Current thread (0x7b8aded0) is n